In [7]:
import math

In [8]:
input = {
        "beam": {
            "h": 500,
            "bf": 300,
            "tf": 20,
            "tw": 8,
            "z": 3619867.15,
            "fy": 240,
            "fu": 340
        },
        "column": {
            "d": 400,
            "t": 20,
            "fy": 240,
            "fu": 340,
            "z": 0
        },
        "loadWidth": 2,
        "beamLength": 8,
        "deadLoad": 5.7,
        "deadLoadCoef": 1.2,
        "liveLoad": 6,
        "liveLoadCoef": 1
    }


In [21]:
beam = IBeam(input["beam"])
column = TubeColumn(input["column"])
BEAM_LENGTH = input["beamLength"]

In [22]:
live = input["liveLoad"]
dead = input["deadLoad"]
loadWidth = input["loadWidth"]

q = load_comb(dead, live)
qu = q * loadWidth

print("qu =", qu , "kn/m")

qu = 25.68 kn/m


In [24]:
RY = 1.15
MAX_CPR = 1.2

db = beam.height + 2 * beam.flangeThickness
lp = db + 10
sh = lp / 1000  # M
lh = BEAM_LENGTH - column.depth/1000 - 2 * sh  # M

pu = 205.0 #JUST FOR TEST SHOULD REMOVE
actual_cpr = (beam.Fy + beam.Fu) / 2 * beam.Fy
cpr = min(actual_cpr , MAX_CPR)
mpr = cpr * RY * (beam.Z / 10**6) * beam.Fy
vpr = (2 * mpr) / lh + (qu * lh + pu) / 2

vu = (vpr + qu * sh) * 10**3
mu = (mpr + vpr * sh + (qu * sh**2)) * 10**6

print("Vu:", round(vu / 10**3 , 2) , "KN")
print("Mu:", round(mu / 10**6 , 2),  "KN.m")

Vu: 568.98 KN
Mu: 1511.84 KN.m


In [25]:
def calc_plate_length(a):
    return mu / (db * 140 * a)  

In [26]:
bpb = beam.flangeWidth + 50
tpb = calc_tp(mu, db, bpb, beam.Fy)
aMin = calculate_welding(beam.flangeThickness)
aMax = min(beam.flangeThickness, tpb) - 2

Lw = calc_plate_length(aMax)
lpb = (Lw / 2) + 20
lpb = round_to_nearest_number(lpb, 25)

print("Bottom Plate Details")
print("Width:", bpb , "mm")
print("thickness:", tpb , "mm")
print("length:", lpb , "mm")
print("a:", aMax , "mm")

Bottom Plate Details
Width: 350 mm
thickness: 35.0 mm
length: 600.0 mm
a: 18 mm


In [28]:
BETA = 0.75
bpt = beam.flangeWidth - 50
aMax = min(beam.flangeThickness, tpb)


In [15]:
def calc_tp(mu, d, b, fy):
    def test(value):
       return mu / (1 * (d + value) * bpb * fy)

    first = test(20) #at first assume thinness is 20mm
    second = test(first)

    return round_to_nearest_number(second, 5)

In [16]:
def load_comb (dead, live):
    return input["deadLoadCoef"] * dead +  input["liveLoadCoef"] * live

In [17]:
def round_to_nearest_number(value, target):
    remainder = value % target
    return value + (target - remainder)

In [18]:
def calculate_welding(thickness):
    if thickness <= 6:
        return 3
    elif thickness <= 12:
        return 5
    elif thickness <= 20:
        return 6
    else:
        return 8

In [19]:
class IBeam:
    def __init__(self, beamInput):
        self.height = beamInput["h"]
        self.flangeWidth = beamInput["bf"]
        self.flangeThickness = beamInput["tf"]
        self.webThickness = beamInput["tw"]
        self.Z = beamInput["z"]
        self.Fy = beamInput["fy"]
        self.Fu = beamInput["fu"]

In [20]:
class TubeColumn:
    def __init__(self, columnInput):
        self.depth = columnInput["d"]
        self.thickness = columnInput["t"]
        self.Z = columnInput["z"]
        self.Fy = columnInput["fy"]
        self.Fu = columnInput["fu"]
